In [45]:
# assign some values to the vector X
grades_array = ["A", "B", "C", "D", "E"]
# Write a grade mutating function
function topstudent!(x)
	x[1:end] .= "AAAA"
end

println("Your previous grades were $(grades_array[4])")

# Call the function on grades_array
topstudent!(grades_array)

println("Your new grades are $(grades_array)")

Your previous grades were D
Your new grades are ["AAAA", "AAAA", "AAAA", "AAAA", "AAAA"]


In [46]:
# non mutating function, more expensive

grades_array2 = ["A", "B", "C", "D", "E"]

function topstudent(x)
	x[4] = "AAAA"
    return x # this is a new copy of x
end

println("Your previous grades were $(grades_array2[4])")

grades_array2 = topstudent(grades_array2)

println("Your new grades are $(grades_array2[4])")


Your previous grades were D
Your new grades are AAAA


In [47]:
# Write negative function for Bool data type
function negative(x::Bool)
	if x == true
    	return false
    else
    	return true
    end
end

# Test the function
println(negative(true))
println(negative(false))



false
true


In [48]:
import Statistics

x = [1, 2, 3, 4, 5]


# Write a function to calculate the mean of a vector
function median(x)
    return Statistics.median(x)
end

# Test the function
println(median(x))





3.0


In [49]:
using CSV
using DataFrames
using Statistics

data = CSV.File("data.csv")

df_people = DataFrame(data)

println(df_people)



weights = df_people[:, 4]
println(weights)

third_age = df_people[3, "age"]

df_by_age_desc = sort(df_people, :age, rev=true)

println(df_by_age_desc[1:5, :])

# Calculate an array of the standardized weights using the array operations .- and ./.

normalized_weights = (df_people.weight .- mean(df_people.weight)) ./ std(df_people.weight)

println(normalized_weights)






10×4 DataFrame
 Row │ name     age    gender   weight 
     │ String7  Int64  String7  Int64  
─────┼─────────────────────────────────
   1 │ John        25  male         70
   2 │ Jane        30  female       65
   3 │ Jim         35  male         75
   4 │ Jill        40  female       60
   5 │ Jack        45  male         72
   6 │ Jill        50  female       63
   7 │ Jim         55  male         77
   8 │ Jane        60  female       66
   9 │ John        65  male         73
  10 │ Jill        70  female       67
[70, 65, 75, 60, 72, 63, 77, 66, 73, 67]
5×4 DataFrame
 Row │ name     age    gender   weight 
     │ String7  Int64  String7  Int64  
─────┼─────────────────────────────────
   1 │ Jill        70  female       67
   2 │ John        65  male         73
   3 │ Jane        60  female       66
   4 │ Jim         55  male         77
   5 │ Jill        50  female       63
[0.21844274129022023, -0.6917353474190285, 1.128620829999469, -1.6019134361282774, 0.5825139767739198, -1

In [50]:
df_big_people = filter(row -> row.weight > 70, df_people)
println(df_big_people)





4×4 DataFrame
 Row │ name     age    gender   weight 
     │ String7  Int64  String7  Int64  
─────┼─────────────────────────────────
   1 │ Jim         35  male         75
   2 │ Jack        45  male         72
   3 │ Jim         55  male         77
   4 │ John        65  male         73


In [63]:
# Read the CSV files directly into DataFrames
old_df = DataFrame(CSV.File("1.csv"))
new_df = DataFrame(CSV.File("2.csv"))

println("Rows in new_df: ", nrow(new_df))
println("Rows in old_df: ", nrow(old_df))

# Group old_df by the compound key and take the first Budget Line Item for each group
old_df_unique = combine(
    groupby(old_df, ["Document", "Description", "Amount"], skipmissing=false),
    "Budget Line Item" => first => "Old Budget Line Item"
)

println("Unique compound keys in old_df: ", nrow(old_df_unique))

# Perform the merge
merged_df = leftjoin(new_df, old_df_unique, 
    on = ["Document", "Description", "Amount"],
    matchmissing = :equal)

# Update Budget Line Item where needed
for i in 1:nrow(merged_df)
    if ismissing(merged_df[i, "Budget Line Item"]) || merged_df[i, "Budget Line Item"] == "-"
        if !ismissing(merged_df[i, "Old Budget Line Item"])
            merged_df[i, "Budget Line Item"] = merged_df[i, "Old Budget Line Item"]
        end
    end
end

# Remove the temporary column
select!(merged_df, Not("Old Budget Line Item"))

# Sort by ITEM NUMBER
sorted_df = sort(merged_df, "ITEM NUMBER")

# dump the ninth empty column from the sorted_df
sorted_df = sorted_df[:, 1:8]

println("Final row count: ", nrow(sorted_df))

println(sorted_df)

#output the sorted_df to a csv file
CSV.write("sorted_df.csv", sorted_df)






Rows in new_df: 351
Rows in old_df: 320
Unique compound keys in old_df: 210
Final row count: 351
351×8 DataFrame
 Row │ ITEM NUMBER  Document  Budget Line Item  Document Date  Label              Description                        Amount       Account       
     │ Int64        Int64     String15?         String15       String31?          String                             String15?    String15      
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │        1001         1  45                5/26/2023      PO20607            StormWind LLC                      $9,900.00    30-37400-5505
   2 │        1002         2  6                 9/30/2023      PPE 09/02/23       Salaries                           $995.45      30-37400-5101
   3 │        1003         2  9                 9/30/2023      PPE 09/02/23       Social Security                    $61.72       30-37400-5121
   4 │        1004  

"sorted_df.csv"